In [1]:
#Mount Drive and install dependencies if running in Colab
def install_dependecies():
  !pip install transformers
  !pip install pytorch-lightning

from sys import path
import os
import sys

if 'google.colab' in str(get_ipython()):
  from google.colab import drive

  root_PATH = '/content/drive/My Drive/nlp-seminar/repository'
  drive_mount_location = '/content/drive'
  module_path = root_PATH + '/src'
  
  drive.mount(drive_mount_location, force_remount=True)
  path.append(root_PATH)

  install_dependecies()
else:
  root_PATH = os.path.abspath("../../..")
  module_path = os.path.abspath(os.path.join('../../../src'))

%load_ext autoreload
%autoreload 2

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from data_processor import DataProcessor
from model_evaluator import ModelEvaluator
from custom_dataset import CustomDataset
from models.bert_custom_model import BERTCustomModel

import pandas as pd
from torch import cuda
from transformers import BertTokenizer, BertModel, BertConfig
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import transformers
import numpy as np
from sklearn import metrics
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import accuracy_score
import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [3]:
# Script configuration
MAX_LEN = 200
TEST_BATCH_SIZE = 32

model_to_use = 'bert-base-uncased'
model_path = root_PATH + '/notebooks/deep_learning_methods/BERT/tb_logs/my_BERT_model/version_0/checkpoints/epoch=3-step=106.ckpt'

#Model parameters
model_params = {
                'test_batch_size': TEST_BATCH_SIZE
                }

gpus_to_use = [0]

In [4]:
#Load datasets
test_df = pd.read_csv(root_PATH + '/data/test.csv')

#Get boolean mask of the datasets
test_boolean_mask = test_df.iloc[:,8:]

#Get topics present in the dataset
remaining_topics = test_boolean_mask.columns.tolist()

#Create new column with the boolean mask
test_df["list"] = test_boolean_mask.values.tolist()

#Remove columns not necessary from the datasets
test_df = test_df[["conversation","list"]].reset_index()

In [5]:
tokenizer = BertTokenizer.from_pretrained(model_to_use)

testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)
data_loader = DataLoader(testing_set, batch_size=TEST_BATCH_SIZE, shuffle= False)

In [6]:
model = BERTCustomModel.load_from_checkpoint(
    model_path, 
    hparams = model_params, 
    training_dataset=None, 
    validation_dataset=None, 
    labels=remaining_topics, 
    model_to_use=model_to_use
    )

trainer = pl.Trainer(gpus=gpus_to_use,
                             #accelerator='dp',
                             #limit_train_batches=32,
                             #limit_val_batches=32,
                             #limit_test_batches=60,
                             #max_epochs=200,
                             #logger=logger,
                             #default_root_dir=logs_path
                             )

trainer.test(ckpt_path=model_path,model=model,test_dataloaders=[data_loader])

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Testing:   1%|          | 32/4910 [00:32<1:21:51,  1.01s/it]

KeyboardInterrupt: 